# Market Sharing

## Objective and Prerequisites

In this example, we’ll show you how to solve a goal programming problem that involves allocating the retailers to two divisions of a company in order to optimize the trade-offs of several market sharing goals. You’ll learn how to create a mixed integer linear programming model of the problem using the Gurobi Python API and how to find an optimal solution to the problem using the Gurobi Optimizer.

This model is example 13 from the fifth edition of Model Building in Mathematical Programming by H. Paul Williams on pages 267-268 and 322-324.

## Problem Description

A large company has two divisions: D1 and D2. The company supplies retailers with oil and spirit. The goal is to allocate each retailer to either division D1 or division D2. The allocated division will be the retailer’s supplier. As far as possible, the allocation must be made so that D1 controls 40% of the market and D2 the remaining 60%. The retailers in the table below are listed as M1 to M23. Each retailer has an estimated market for oil and spirit. Retailers M1 to M8 are in region 1, retailers M9 to M18 are in region 2, and retailers M19 to M23 are in region 3. Certain retailers are considered to have good growth prospects and categorized as group A and the others are in group B. Each retailer has a certain number of delivery points. 

|Retailer    |Retailer   |Oil Market |Delivery Points |Spirit Market |Growth Cat |
|------------|-----------|-----------|----------------|--------------|-----------|
|Region 1    |M1         |9          |11              |34            |A          |
|Region 1    |M2         |13         |47              |411           |A          |
|Region 1    |M3         |14         |44              |82            |A          |
|Region 1    |M4         |17         |25              |157           |B          |
|Region 1    |M5         |18         |10              |5             |A          |
|Region 1    |M6         |19         |26              |183           |A          |
|Region 1    |M7         |23         |26              |14            |B          |
|Region 1    |M8         |21         |54              |215           |B          |
|Region 2    |M9         |9          |18              |102           |B          |
|Region 2    |M10        |11         |51              |21            |A          |
|Region 2    |M11        |17         |20              |54            |B          |
|Region 2    |M12        |18         |105             |0             |B          |
|Region 2    |M13        |18         |7               |6             |B          |
|Region 2    |M14        |17         |16              |96            |B          |
|Region 2    |M15        |22         |34              |118           |A          |
|Region 2    |M16        |24         |100             |112           |B          |
|Region 2    |M17        |36         |50              |535           |B          |
|Region 2    |M18        |43         |21              |8             |B          |
|Region 3    |M19        |6          |11              |53            |B          |
|Region 3    |M20        |15         |19              |28            |A          |
|Region 3    |M21        |15         |14              |69            |B          |
|Region 3    |M22        |25         |10              |65            |B          |
|Region 3    |M23        |39         |11              |27            |B          |


We want to make the 40%/60% split between D1 and D2 in each of the following categories:
1. Total number of delivery points
2. Control of spirit market
3. Control of oil market in region 1
4. Control of oil market in region 2
5. Control of oil market in region 3
6. Number of retailers in group A
7. Number of retailers in group B.

There is flexibility in that any market share may vary by $\pm$ 5%. That is, the share can vary between the limits 35%/65% and 45%/55%. The objective is to minimize the sum of the percentage deviations from the 40%/60% split.

## Model Formulation

### Sets and Indices

$r \in \text{Retailers}=\{\ 1,2,...,23\}$

### Parameters

$\text{deliveryPoints}_{r} \in \mathbb{N}^+$: Delivery points of retailer $r$.

$\text{spiritMarket}_{r} \in \mathbb{R}^+$: Spirit market -in millions of gallons, of retailer $r$.

$\text{oilMarket1}_{r} \in \mathbb{R}^+$: Oil market -in millions of gallons of retailer $r$ in Region 1.

$\text{oilMarket2}_{r} \in \mathbb{R}^+$: Oil market -in millions of gallons of retailer $r$ in Region 2.

$\text{oilMarket3}_{r} \in \mathbb{R}^+$: Oil market -in millions of gallons of retailer $r$ in Region 3.

$\text{retailerA}_{r} \in \{0,1\}$: Parameter has a value of 1 if retailer $r$  belongs to group A.

$\text{retailerB}_{r} \in \{0,1\}$: Parameter has a value of 1 if retailer $r$  belongs to group B.

$\text{deliveryPoints40} \in \mathbb{R}^+$: Forty percent of the delivery points.

$\text{deliveryPoints5} \in \mathbb{R}^+$: Five percent of the delivery points.

$\text{spiritMarket40} \in \mathbb{R}^+$: Forty percent of  the spirit market.

$\text{spiritMarket5} \in \mathbb{R}^+$: Five percent of the spirit market.

$\text{oilMarket1_40} \in \mathbb{R}^+$: Forty percent of  the oil market in region 1.

$\text{oilMarket1_5} \in \mathbb{R}^+$: Five percent of  the oil market in region 1.

$\text{oilMarket2_40} \in \mathbb{R}^+$: Forty percent of  the oil market in region 2.

$\text{oilMarket2_5} \in \mathbb{R}^+$: Five percent of  the oil market in region 2.

$\text{oilMarket3_40} \in \mathbb{R}^+$: Forty percent of  the oil market in region 3.

$\text{oilMarket3_5} \in \mathbb{R}^+$: Five percent of  the oil market in region 3.

$\text{retailerA40} \in \mathbb{R}^+$: Forty percent of the number of retailers in group A.

$\text{retailerA5} \in \mathbb{R}^+$: Five percent of the number of retailers in group A.

$\text{retailerB40} \in \mathbb{R}^+$: Forty percent of the number of retailers in group B.

$\text{retailerB5} \in \mathbb{R}^+$: Five percent of the number of retailers in group B.

### Decision Variables

$\text{allocate}_{r} \in \{0,1\}$: This binary variable is equal 1, if retailer r is allocated to Division 1, and 0 if allocated to Division 2.

$\text{deliveryPointsPos} \in \mathbb{R}^+$: This decision variable measures the positive deviation of the retailers’ allocation for the goal of satisfying forty percent of the delivery points.

$\text{deliveryPointsNeg} \in \mathbb{R}^+$: This decision variable measures the negative deviation of the retailers’ allocation for the goal of satisfying forty percent of the delivery points.

$\text{spiritMarketPos} \in \mathbb{R}^+$: This decision variable measures the positive deviation of the retailers’ allocation for the goal of satisfying forty percent  of the spirit market.

$\text{spiritMarketNeg} \in \mathbb{R}^+$: This decision variable measures the negative deviation of the retailers’ allocation for the goal of satisfying forty percent  of the spirit market.

$\text{oilMarket1Pos} \in \mathbb{R}^+$: This decision variable measures the positive deviation of the retailers’ allocation for the goal of satisfying forty percent  of  the oil market in region 1.

$\text{oilMarket1Neg} \in \mathbb{R}^+$: This decision variable measures the negative deviation of the retailers’ allocation for the goal of satisfying forty percent  of  the oil market in region 1.

$\text{oilMarket2Pos} \in \mathbb{R}^+$: This decision variable measures the positive deviation of the retailers’ allocation for the goal of satisfying forty percent  of  the oil market in region 2.

$\text{oilMarket2Neg} \in \mathbb{R}^+$: This decision variable measures the negative deviation of the retailers’ allocation for the goal of satisfying forty percent  of the oil market in region 2.

$\text{oilMarket3Pos} \in \mathbb{R}^+$: This decision variable measures the positive deviation of the retailers’ allocation for the goal of satisfying forty percent  of  the oil market in region 3.

$\text{oilMarket3Neg} \in \mathbb{R}^+$: This decision variable measures the negative deviation of the retailers’ allocation for the goal of satisfying forty percent  of the oil market in region 3.

$\text{retailerAPos} \in \mathbb{R}^+$: This decision variable measures the positive deviation of the retailers’ allocation for the goal of satisfying forty percent of the number of retailers in group A.

$\text{retailerANeg} \in \mathbb{R}^+$: This decision variable measures the negative deviation of the retailers’ allocation for the goal of satisfying forty percent of the number of retailers in group A.

$\text{retailerBPos} \in \mathbb{R}^+$: This decision variable measures the positive deviation of the retailers’ allocation for the goal of satisfying forty percent of the number of retailers in group B.

$\text{retailerBNeg} \in \mathbb{R}^+$: This decision variable measures the negative deviation of the retailers’ allocation for the goal of satisfying forty percent of the number of retailers in group B.

### Constraints

**Delivery points**: The allocation of retailers at Division 1 satisfies as much as possible forty percent of the delivery points.

\begin{equation}
\sum_{r \in \text{Retailers}} \text{deliveryPoints}_{r}*{\text{allocate}_{r}} + \text{deliveryPointsPos} - \text{deliveryPointsNeg}  = \text{deliveryPoints40}
\end{equation}

**Spirit Market**: The allocation of retailers at Division 1 satisfies as much as possible forty percent of the spirit market.

\begin{equation}
\sum_{r \in \text{Retailers}} \text{spiritMarket}_{r}*{\text{allocate}_{r}} + \text{spiritMarketPos} - 
\text{spiritMarketNeg}  = \text{spiritMarket40}
\end{equation}

**Oil market region 1**: The allocation of retailers in region 1 at Division 1 satisfies as much as possible forty percent of the oil market in that region.

\begin{equation}
\sum_{r \in \text{Retailers}} \text{oilMarket1}_{r}*{\text{allocate}_{r}} + \text{oilMarket1Pos} - 
\text{oilMarket1Neg}  = \text{oilMarket1_40}
\end{equation}

**Oil market region 2**: The allocation of retailers in region 2 at Division 1 satisfies as much as possible forty percent of the oil market in that region.

\begin{equation}
\sum_{r \in \text{Retailers}} \text{oilMarket2}_{r}*{\text{allocate}_{r}} + \text{oilMarket2Pos} - 
\text{oilMarket2Neg}  = \text{oilMarket2_40}
\end{equation}

**Oil market region 3**: The allocation of retailers in region 3 at Division 1 satisfies as much as possible forty percent of the oil market in that region.

\begin{equation}
\sum_{r \in \text{Retailers}} \text{oilMarket3}_{r}*{\text{allocate}_{r}} + \text{oilMarket3Pos} - 
\text{oilMarket3Neg}  = \text{oilMarket3_40}
\end{equation}

**Group A**: The allocation of retailers at Division 1 satisfies as much as possible forty percent of the retailers in group A.

\begin{equation}
\sum_{r \in \text{Retailers}} \text{retailerA40}_{r}*{\text{allocate}_{r}} + \text{retailerAPos} - 
\text{retailerANeg}  = \text{retailerA40}
\end{equation}

**Group B**: The allocation of retailers at Division 1 satisfies as much as possible forty percent of the retailers in group B.

\begin{equation}
\sum_{r \in \text{Retailers}} \text{retailerB40}_{r}*{\text{allocate}_{r}} + \text{retailerBPos} - 
\text{retailerBNeg}  = \text{retailerB40}
\end{equation}

**Flexibility**: There is flexibility in that any market share may vary by $\pm$ 5%.

$$
\text{deliveryPointsPos} \leq \text{deliveryPoints5}
$$

$$
\text{deliveryPointsNeg}  \leq \text{deliveryPoints5}
$$

$$
\text{spiritMarketPos} \leq \text{spiritMarket5}
$$

$$
\text{spiritMarketNeg}  \leq \text{spiritMarket5}
$$

$$
\text{oilMarket1Pos} \leq \text{oilMarket1_5}
$$

$$
\text{oilMarket1Neg} \leq \text{oilMarket1_5}
$$

$$
\text{oilMarket2Pos} \leq \text{oilMarket2_5}
$$

$$
\text{oilMarket2Neg} \leq \text{oilMarket2_5}
$$

$$
\text{oilMarket3Pos} \leq \text{oilMarket3_5}
$$

$$
\text{oilMarket3Neg} \leq \text{oilMarket3_5}
$$

$$
\text{retailerAPos} \leq \text{retailerA5}
$$

$$
\text{retailerANeg} \leq \text{retailerA5}
$$

$$
\text{retailerBPos} \leq \text{retailerB5}
$$

$$
\text{retailerBNeg} \leq \text{retailerB5}
$$

### Objective Function

**Minimize deviations**: Minimize the sum of positive and negative deviations.

\begin{equation}
\text{Minimize} \quad  \text{deliveryPointsPos} + \text{deliveryPointsNeg} + \text{spiritMarketPos} + \text{spiritMarketNeg} +
\text{oilMarket1Pos} + \text{oilMarket1Neg}
\end{equation}

$$
+ \text{oilMarket2Pos} + \text{oilMarket2Neg} + \text{oilMarket3Pos} + \text{oilMarket3Neg} 
$$

$$
+ \text{retailerAPos} + \text{retailerANeg} + \text{retailerBPos} + \text{retailerBNeg}
$$

## Python Implementation

We import the Gurobi Python Module and other Python libraries.

In [ ]:
%pip install gurobipy

In [1]:
import numpy as np
import pandas as pd
from itertools import product

import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.0

## Input data

We define all the input data for the model.

In [2]:
# Create a dictionary to capture the delivery points and spirit market -in millions of gallons.

retailers, deliveryPoints, spiritMarket = gp.multidict(
    {
        (1): [11, 34],
        (2): [47, 411],
        (3): [44, 82],
        (4): [25, 157],
        (5): [10, 5],
        (6): [26, 183],
        (7): [26, 14],
        (8): [54, 215],
        (9): [18, 102],
        (10): [51, 21],
        (11): [20, 54],
        (12): [105, 0],
        (13): [7, 6],
        (14): [16, 96],
        (15): [34, 118],
        (16): [100, 112],
        (17): [50, 535],
        (18): [21, 8],
        (19): [11, 53],
        (20): [19, 28],
        (21): [14, 69],
        (22): [10, 65],
        (23): [11, 27],
    }
)

# Create a dictionary to capture the oil market -in millions of gallons for region 1.

retailers1, oilMarket1 = gp.multidict(
    {(1): 9, (2): 13, (3): 14, (4): 17, (5): 18, (6): 19, (7): 23, (8): 21}
)

# Create a dictionary to capture the oil market -in millions of gallons for region 2.

retailers2, oilMarket2 = gp.multidict(
    {
        (9): 9,
        (10): 11,
        (11): 17,
        (12): 18,
        (13): 18,
        (14): 17,
        (15): 22,
        (16): 24,
        (17): 36,
        (18): 43,
    }
)

# Create a dictionary to capture the oil market -in millions of gallons for region 3.

retailers3, oilMarket3 = gp.multidict({(19): 6, (20): 15, (21): 15, (22): 25, (23): 39})

# Create a dictionary to capture retailers in group A.

groupA, retailerA = gp.multidict(
    {(1): 1, (2): 1, (3): 1, (5): 1, (6): 1, (10): 1, (15): 1, (20): 1}
)

# Create a dictionary to capture retailers in group B.

groupB, retailerB = gp.multidict(
    {
        (4): 1,
        (7): 1,
        (8): 1,
        (9): 1,
        (11): 1,
        (12): 1,
        (13): 1,
        (14): 1,
        (16): 1,
        (17): 1,
        (18): 1,
        (19): 1,
        (21): 1,
        (22): 1,
        (23): 1,
    }
)

# Forty and five percentages of each goal

deliveryPoints40 = 292
deliveryPoints5 = 36.5
spiritMarket40 = 958
spiritMarket5 = 119.75
oilMarket1_40 = 53.6
oilMarket1_5 = 6.7
oilMarket2_40 = 86
oilMarket2_5 = 10.75
oilMarket3_40 = 40
oilMarket3_5 = 5
retailerA40 = 3.2
retailerA5 = 0.4
retailerB40 = 6
retailerB5 = 0.75

## Model Deployment

We create a model and the variables. The main decision variable is a binary variable that is equal to 1  when a retailer is allocated to Division 1, and 0 when allocated it to Division 2. The rest of the decision variables measure positive and negative deviations from the seven goals of the 40%/60% split.

In [3]:
model = gp.Model("MarketSharing")

# Allocation of retailers to Division 1.
allocate = model.addVars(retailers, vtype=GRB.BINARY, name="allocate")

# Positive and negative deviation of delivery points goal.

deliveryPointsPos = model.addVar(ub=deliveryPoints5, name="deliveryPointsPos")
deliveryPointsNeg = model.addVar(ub=deliveryPoints5, name="deliveryPointsNeg")

# Positive and negative deviation of spirit market goal.

spiritMarketPos = model.addVar(ub=spiritMarket5, name="spiritMarketPos")
spiritMarketNeg = model.addVar(ub=spiritMarket5, name="spiritMarketNeg")

# Positive and negative deviation of oil market in region 1 goal.

oilMarket1Pos = model.addVar(ub=oilMarket1_5, name="oilMarket1Pos")
oilMarket1Neg = model.addVar(ub=oilMarket1_5, name="oilMarket1Neg")

# Positive and negative deviation of oil market in region 2 goal.

oilMarket2Pos = model.addVar(ub=oilMarket2_5, name="oilMarket2Pos")
oilMarket2Neg = model.addVar(ub=oilMarket2_5, name="oilMarket2Neg")

# Positive and negative deviation of oil market in region 3 goal.

oilMarket3Pos = model.addVar(ub=oilMarket3_5, name="oilMarket3Pos")
oilMarket3Neg = model.addVar(ub=oilMarket3_5, name="oilMarket3Neg")

# Positive and negative deviation of retailers in group A goal.

retailerAPos = model.addVar(ub=retailerA5, name="retailerAPos")
retailerANeg = model.addVar(ub=retailerA5, name="retailerANeg")

# Positive and negative deviation of retailers in group B goal.

retailerBPos = model.addVar(ub=retailerB5, name="retailerBPos")
retailerBNeg = model.addVar(ub=retailerB5, name="retailerBNeg")

Using license file c:\gurobi\gurobi.lic


The allocation of retailers at Division 1 satisfies as much as possible forty percent of the delivery points.

In [4]:
# Delivery points constraint.

DPConstr = model.addConstr(
    (
        gp.quicksum(deliveryPoints[r] * allocate[r] for r in retailers)
        + deliveryPointsPos
        - deliveryPointsNeg
        == deliveryPoints40
    ),
    name="DPConstrs",
)

The allocation of retailers at Division 1 satisfies as much as possible forty percent of the spirit market.

In [5]:
# Spirit market constraint.

SMConstr = model.addConstr(
    (
        gp.quicksum(spiritMarket[r] * allocate[r] for r in retailers)
        + spiritMarketPos
        - spiritMarketNeg
        == spiritMarket40
    ),
    name="SMConstr",
)

The allocation of retailers in region 1 at Division 1 satisfies as much as possible forty percent of the oil market in that region.

In [6]:
# Oil market in region 1 constraint.

OM1Constr = model.addConstr(
    (
        gp.quicksum(oilMarket1[r] * allocate[r] for r in retailers1)
        + oilMarket1Pos
        - oilMarket1Neg
        == oilMarket1_40
    ),
    name="OM1Constr",
)

The allocation of retailers in region 2 at Division 1 satisfies as much as possible forty percent of the oil market in that region.

In [7]:
# Oil market in region 2 constraint.

OM2Constr = model.addConstr(
    (
        gp.quicksum(oilMarket2[r] * allocate[r] for r in retailers2)
        + oilMarket2Pos
        - oilMarket2Neg
        == oilMarket2_40
    ),
    name="OM2Constr",
)

The allocation of retailers in region 3 at Division 1 satisfies as much as possible forty percent of the oil market in that region.

In [8]:
# Oil market in region 3 constraint.

OM3Constr = model.addConstr(
    (
        gp.quicksum(oilMarket3[r] * allocate[r] for r in retailers3)
        + oilMarket3Pos
        - oilMarket3Neg
        == oilMarket3_40
    ),
    name="OM3Constr",
)

The allocation of retailers at Division 1 satisfies as much as possible forty percent of the retailers in group A.

In [9]:
# Group A constraint.

AConstr = model.addConstr(
    (
        gp.quicksum(retailerA[r] * allocate[r] for r in groupA)
        + retailerAPos
        - retailerANeg
        == retailerA40
    ),
    name="AConstr",
)

The allocation of retailers at Division 1 satisfies as much as possible forty percent of the retailers in group B.

In [10]:
# Group B constraint.

BConstr = model.addConstr(
    (
        gp.quicksum(retailerB[r] * allocate[r] for r in groupB)
        + retailerBPos
        - retailerBNeg
        == retailerB40
    ),
    name="BConstr",
)

Minimize the sum of positive and negative deviations.

In [11]:
# Objective function

obj = (
    deliveryPointsPos
    + deliveryPointsNeg
    + spiritMarketPos
    + spiritMarketNeg
    + oilMarket1Pos
    + oilMarket1Neg
    + oilMarket2Pos
    + oilMarket2Neg
    + oilMarket3Pos
    + oilMarket3Neg
    + retailerAPos
    + retailerANeg
    + retailerBPos
    + retailerBNeg
)

model.setObjective(obj)

In [12]:
# Verify model formulation

model.write("marketSharing.lp")

# Run optimization engine

model.optimize()

Gurobi Optimizer version 9.1.0 build v9.1.0rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 7 rows, 37 columns and 105 nonzeros
Model fingerprint: 0xa5aab3a9
Variable types: 14 continuous, 23 integer (23 binary)
Coefficient statistics:
  Matrix range     [1e+00, 5e+02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [4e-01, 1e+02]
  RHS range        [3e+00, 1e+03]
Presolve time: 0.00s
Presolved: 7 rows, 37 columns, 105 nonzeros
Variable types: 14 continuous, 23 integer (23 binary)

Root relaxation: objective 0.000000e+00, 13 iterations, 0.00 seconds

    Nodes    |    Current Node    |     Objective Bounds      |     Work
 Expl Unexpl |  Obj  Depth IntInf | Incumbent    BestBd   Gap | It/Node Time

     0     0    0.00000    0    7          -    0.00000      -     -    0s
H    0     0                     131.6000000    0.00000   100%     -    0s
H    0     0                     111.6000000    0.00000   100%     -    0s
H

## Analysis

The allocation of retailers to Division 1 that minimizes the sum of positive and negative deviations from the goal follows. In addition, we show how each goal is within the 35%/45% range values.

In [13]:
# Output reports

print(
    "\n\n_________________________________________________________________________________"
)
print(f"The optimal allocation of retailers to Division 1 is:")
print(
    "_________________________________________________________________________________"
)
for r in retailers:
    if allocate[r].x > 0.5:
        print(f"Retailer{r}")

# print(f"\nThe optimal objective function value is {model.objVal}")



_________________________________________________________________________________
The optimal allocation of retailers to Division 1 is:
_________________________________________________________________________________
Retailer2
Retailer6
Retailer7
Retailer9
Retailer12
Retailer13
Retailer14
Retailer15
Retailer23


The following report validates that the goals have been satisfied within acceptable 35% and 45% ranges.

In [14]:
# Test that the solution is within acceptable ranges.

goal_ranges = pd.DataFrame(columns=["Goal", "Min_35", "Actual", "Max_45"])

count = 0

DeliveryPointsGoal = 0
for r in retailers:
    if allocate[r].x > 0.5:
        DeliveryPointsGoal += deliveryPoints[r] * allocate[r].x

goal_ranges = goal_ranges.append(
    {
        "Goal": "Delivery points",
        "Min_35": round(deliveryPoints40 * (0.35 / 0.40), 2),
        "Actual": round(DeliveryPointsGoal, 2),
        "Max_45": round(deliveryPoints40 * (0.45 / 0.40), 2),
    },
    ignore_index=True,
)
count += 1

spiritMarketGoal = 0
for r in retailers:
    if allocate[r].x > 0.5:
        spiritMarketGoal += spiritMarket[r] * allocate[r].x

goal_ranges = goal_ranges.append(
    {
        "Goal": "Spirit market",
        "Min_35": round(spiritMarket40 * (0.35 / 0.40), 2),
        "Actual": round(spiritMarketGoal, 2),
        "Max_45": round(spiritMarket40 * (0.45 / 0.40), 2),
    },
    ignore_index=True,
)
count += 1

oilMarket1Goal = 0
for r in retailers1:
    if allocate[r].x > 0.5:
        oilMarket1Goal += oilMarket1[r] * allocate[r].x

goal_ranges = goal_ranges.append(
    {
        "Goal": "Oil market1",
        "Min_35": round(oilMarket1_40 * (0.35 / 0.40), 2),
        "Actual": round(oilMarket1Goal, 2),
        "Max_45": round(oilMarket1_40 * (0.45 / 0.40), 2),
    },
    ignore_index=True,
)
count += 1

oilMarket2Goal = 0
for r in retailers2:
    if allocate[r].x > 0.5:
        oilMarket2Goal += oilMarket2[r] * allocate[r].x
#
goal_ranges = goal_ranges.append(
    {
        "Goal": "Oil market2",
        "Min_35": round(oilMarket2_40 * (0.35 / 0.40), 2),
        "Actual": round(oilMarket2Goal, 2),
        "Max_45": round(oilMarket2_40 * (0.45 / 0.40), 2),
    },
    ignore_index=True,
)
count += 1

oilMarket3Goal = 0
for r in retailers3:
    if allocate[r].x > 0.5:
        oilMarket3Goal += oilMarket3[r] * allocate[r].x
#
goal_ranges = goal_ranges.append(
    {
        "Goal": "Oil market3",
        "Min_35": round(oilMarket3_40 * (0.35 / 0.40), 2),
        "Actual": round(oilMarket3Goal, 2),
        "Max_45": round(oilMarket3_40 * (0.45 / 0.40), 2),
    },
    ignore_index=True,
)
count += 1

retailerAGoal = 0
for r in groupA:
    if allocate[r].x > 0.5:
        retailerAGoal += retailerA[r] * allocate[r].x
#
goal_ranges = goal_ranges.append(
    {
        "Goal": "Group A",
        "Min_35": round(retailerA40 * (0.35 / 0.40), 2),
        "Actual": round(retailerAGoal, 2),
        "Max_45": round(retailerA40 * (0.45 / 0.40), 2),
    },
    ignore_index=True,
)
count += 1

retailerBGoal = 0
for r in groupB:
    if allocate[r].x > 0.5:
        retailerBGoal += retailerB[r] * allocate[r].x
#
goal_ranges = goal_ranges.append(
    {
        "Goal": "Group B",
        "Min_35": round(retailerB40 * (0.35 / 0.40), 2),
        "Actual": round(retailerBGoal, 2),
        "Max_45": round(retailerB40 * (0.45 / 0.40), 2),
    },
    ignore_index=True,
)
count += 1

goal_ranges.index = [""] * count
goal_ranges

Goal  Min_35  Actual   Max_45
  Delivery points  255.50   290.0   328.50
    Spirit market  838.25   957.0  1077.75
      Oil market1   46.90    55.0    60.30
      Oil market2   75.25    84.0    96.75
      Oil market3   35.00    39.0    45.00
          Group A    2.80     3.0     3.60
          Group B    5.25     6.0     6.75

## References

H. Paul Williams, Model Building in Mathematical Programming, fifth edition.

Copyright © 2020 Gurobi Optimization, LLC